In [62]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree
from sklearn.ensemble import BaggingClassifier

In [47]:
df = pd.read_csv("card_credit_fraud.csv")
df['type'] = df['type'].replace(["CASH_OUT","PAYMENT","TRANSFER","DEBIT","CASH_IN"], [0, 1, 2, 3, 4])


df['nameOrig'] = df['nameOrig'].str.replace('C', '0', regex=False)
df['nameOrig'] = df['nameOrig'].str.replace('M', '1', regex=False)
df['nameDest'] = df['nameOrig'].str.replace('C', '0', regex=False)
df['nameDest'] = df['nameOrig'].str.replace('M', '1', regex=False)

# Affichez les premières lignes pour vérifier
df.head()


,transactionId,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,0,1,1,983964,01231006815,170136,16029636,01231006815,0,0,0
1,1,1,1,186428,01666544295,21249,1938472,01666544295,0,0,0
2,2,1,2,181,01305486145,181,0,01305486145,0,0,1
3,3,1,0,181,0840083671,181,0,0840083671,21182,0,1
4,4,1,1,1166814,02048537720,41554,2988586,02048537720,0,0,0


In [48]:
#separation apprentisage/test

y = df['isFraud']
X = df.drop(["isFraud"], axis=1)
print(X)

         transactionId  step  type     amount     nameOrig  oldbalanceOrg  \
0                    0     1     1     983964  01231006815         170136   
1                    1     1     1     186428  01666544295          21249   
2                    2     1     2        181  01305486145            181   
3                    3     1     0        181   0840083671            181   
4                    4     1     1    1166814  02048537720          41554   
...                ...   ...   ...        ...          ...            ...   
1008208        6362615   743     0   33968213   0786484425       33968213   
1008209        6362616   743     2  631140928  01529008245      631140928   
1008210        6362617   743     0  631140928  01162922333      631140928   
1008211        6362618   743     2   85000252  01685995037       85000252   
1008212        6362619   743     0   85000252  01280323807       85000252   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  
0   

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [50]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train,y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [51]:
y_pred = clf.predict(X_test)
print(y_pred) # modèle qui a prédit 

[0 0 0 ... 0 0 0]


In [52]:
#Accurancy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.9995139917477782

In [53]:
#Matrice de confusion
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

print(confusion_matrix(y_test,y_pred))

[[300099      0]
 [   147   2218]]


In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    300099
           1       1.00      0.94      0.97      2365

    accuracy                           1.00    302464
   macro avg       1.00      0.97      0.98    302464
weighted avg       1.00      1.00      1.00    302464



In [55]:
from imblearn.over_sampling import SMOTE
seed = 5
smote = SMOTE(sampling_strategy='auto', random_state=seed, k_neighbors=7)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [56]:
clf.fit(X_train_smote,y_train_smote)

RandomForestClassifier(max_depth=2, random_state=0)

In [57]:
y_pred_smote = clf.predict(X_test)
print(y_pred_smote) # modèle qui a prédit 

[0 0 0 ... 0 0 0]


In [58]:
#Accurancy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred_smote)
accuracy

0.9995139917477782

In [59]:
print(confusion_matrix(y_test,y_pred_smote))

[[300099      0]
 [   147   2218]]


In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_smote))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    300099
           1       1.00      0.94      0.97      2365

    accuracy                           1.00    302464
   macro avg       1.00      0.97      0.98    302464
weighted avg       1.00      1.00      1.00    302464



In [ ]:
# Prédire sur l'ensemble de test avec le meilleur modèle
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Évaluer les performances du modèle sur l'ensemble de test
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Précision sur l'ensemble de test:", accuracy)

In [70]:
# Définir la grille de paramètres à explorer
param_grid = {
    'n_estimators': [30, 60, 90],
    'max_depth': [None, 2, 4],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3]
}

# Créer un objet GridSearchCV avec une métrique de rappel (recall)
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, scoring='recall')

# Exécuter la recherche sur grille sur les données
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation croisée (rappel):", grid_search.best_score_)

# Prédire sur l'ensemble de test avec le meilleur modèle
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Évaluer les performances du modèle sur l'ensemble de test
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred)
print("Rappel sur l'ensemble de test:", recall)


Meilleurs paramètres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 60}
Meilleur score de validation croisée (rappel): 0.9700743313467788
Rappel sur l'ensemble de test: 0.9767441860465116


In [74]:
clf_grid = RandomForestClassifier(max_depth=None,min_samples_leaf=1,min_samples_split=2,n_estimators=60,random_state=0)

clf_grid.fit(X_train_smote,y_train_smote)

y_pred_grid = clf_grid.predict(X_test)
print(y_pred_grid) # modèle qui a prédit 

#Matrice de confusion
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

print(confusion_matrix(y_test,y_pred))

#recall score
recall = recall_score(y_test, y_pred_grid)
print("Rappel est egale a :", recall)

[0 0 0 ... 0 0 0]
[[300098      1]
 [    55   2310]]
Rappel est egale a : 0.9826638477801268


In [76]:
print(classification_report(y_test,y_pred_grid))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    300099
           1       0.99      0.98      0.99      2365

    accuracy                           1.00    302464
   macro avg       0.99      0.99      0.99    302464
weighted avg       1.00      1.00      1.00    302464

